# Load microorganism

In [ ]:
# Add path to the model
import sys
sys.path.append("../")
import cobra
from microorganism import Microorganism

# Read E. coli model from cobrapy
ecoli = cobra.io.load_json_model("../data/original/e_coli_core.json")
# Create a microorganism object
cobra_model = Microorganism(cobra_model=ecoli)
print(cobra_model)

### Create Metabolic Network

In [ ]:
# add folder for imports
import sys
sys.path.append('../')

import pandas as pd
import numpy as np
import json
import networkx as nx
from tqdm import tqdm
from compound import Compound
from reaction import Reaction
from graph import Graph
from data import Data
from pathway import Pathway
from utils import create_compound, create_reaction
# suppres rdkit warnings
import rdkit
from rdkit import RDLogger
from utils import check_list

RDLogger.DisableLog('rdApp.*')


# read data from csv
cpds = pd.read_csv('../data/compounds_final.csv', index_col=0) # containing toxicity
rxns = pd.read_csv('../data/reactions_final.csv', index_col=0)
pairs = pd.read_csv('../data/pairs_final_RPAIRS_pred.csv', index_col=0)
cofactors = pd.read_csv('../data/original/cofactors_KEGG.csv')

# create class instances
data = Data()
graph = Graph(pairs=pairs)
pathway = Pathway()

data = create_compound(data, cpds, cofactors)
data = create_reaction(data, rxns)

# CREATE GRAPH
graph.create_graph(data=data, pairs=pairs)

graph.calculate_edge_mol_weight(data)
graph.calculate_smiles_similarity(data)

#### Get KEGG source and target

In [ ]:
# define target compound
butanol_KEGG = 'C06142'

# define example source compound
oxoglatarate_KEGG = 'C00026'

# define source list
src_list = ['C00084', 'C00024', 'C00074', 'C00022', 'C00497', 'C00149', \
            'C00122', 'C00042', 'C00036', 'C00091', 'C00111', 'C00158']

#### Find metabolic pathways from src to trg

Find metabolic pathways from `Acetyl-coA` to `n-Butanol`

In [ ]:
pathway.initialize(src_list[1], butanol_KEGG, graph)
pathway.get_pathway()
# pathway.print_pathway(with_reactions=True)

Get a single pathway for further research

In [ ]:
pathway.single_pathway(pathway.pred_paths[pathway.idx_smi[0]])

In [ ]:
pathway.select_reactions(data, cobra_model)
print(pathway.selected_pathway_reactions)

In [ ]:
pathway.reactions_add_gem(data, cobra_model, 'metabolites')

In [ ]:
pathway.reactions_add_gem(data, cobra_model, 'seed')